In [20]:
import pandas as pd  
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

import warnings
warnings.filterwarnings('ignore')

In [21]:
import torch
import torch.nn as nn
# Define FFNN Model
class FFNN(nn.Module):
    def __init__(self, input_size, output_size, hiddens=[50, 100, 50]):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hiddens[0])
        self.fc2 = nn.Linear(hiddens[0], hiddens[1])
        self.fc3 = nn.Linear(hiddens[1], hiddens[2])
        self.fc4 = nn.Linear(hiddens[2], output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [22]:
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch

test_data = np.load(f'np_data/test_data_01.npz')
X = test_data['X']
y = test_data['y']
X_test = torch.tensor(X, dtype=torch.float32)
Y_test = torch.tensor(y, dtype=torch.long)
test_dataset = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [23]:
# Load model and test
# hiddens = [50, 100, 50]
# hiddens = [100, 200, 100]
hiddens = [200, 400, 200]
model_path = f'fdia_model_ffnn_pytorch_{hiddens[0]}_{hiddens[1]}_{hiddens[2]}_torch.pth'
model = FFNN(input_size=X.shape[1], output_size=2, hiddens=hiddens)
criterion = nn.CrossEntropyLoss()
model.load_state_dict(torch.load(model_path))
model.to("cuda")

import torch
import torch.nn.functional as F

model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for inputs, labels in test_loader:
        inputs = inputs.to("cuda")
        labels = labels.to("cuda")
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)

print('Test Accuracy: {:.2f} %'.format(accuracy * 100))
print('F1-score: {:.2f}'.format(f1))
print('Precision: {:.2f}'.format(precision))
print('Recall: {:.2f}'.format(recall))


Test Accuracy: 93.72 %
F1-score: 0.86
Precision: 0.87
Recall: 0.84
